In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("unmoved/cure-the-princess")

print("Path to dataset files:", path)

100%|██████████| 49.2k/49.2k [00:00<00:00, 40.3MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/unmoved/cure-the-princess/versions/1


In [5]:
import pandas as pd

file_path = '/root/.cache/kagglehub/datasets/unmoved/cure-the-princess/versions/1/data.csv'
df = pd.read_csv(file_path)
print(df.head())


   Phoenix Feather  Unicorn Horn  Dragon's Blood  Mermaid Tears  Fairy Dust  \
0              2.4          18.7            18.4           27.9         7.9   
1              2.1           6.0            15.0           13.3        15.6   
2             17.2          13.9            23.8            6.8        10.7   
3              8.4           9.7             6.8           26.9         4.6   
4             22.1          10.8            16.4           10.5        22.0   

   Goblin Toes  Witch's Brew  Griffin Claw  Troll Hair  Kraken Ink  \
0          9.6          18.3          13.2         2.5        26.0   
1         13.1          11.0           5.0         7.2        26.0   
2         15.8          19.4           2.7        15.4        21.2   
3         29.1          14.6          19.7        18.0        20.8   
4         23.4           2.6          18.2        23.8        11.3   

   Minotaur Horn  Basilisk Scale  Chimera Fang  Cured  
0           10.5            26.2          12.5  

In [7]:
print(df.isnull().sum())
print(df['Cured'].value_counts())

Phoenix Feather    0
Unicorn Horn       0
Dragon's Blood     0
Mermaid Tears      0
Fairy Dust         0
Goblin Toes        0
Witch's Brew       0
Griffin Claw       0
Troll Hair         0
Kraken Ink         0
Minotaur Horn      0
Basilisk Scale     0
Chimera Fang       0
Cured              0
dtype: int64
Cured
0    1177
1    1161
Name: count, dtype: int64


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop('Cured', axis=1).values
y = df['Cured'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
import torch

X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test).unsqueeze(1)

In [11]:
import torch.nn as nn

class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

input_dim = X_train.shape[1]
model = LogisticRegressionModel(input_dim)

In [12]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 0.7868
Epoch [20/100], Loss: 0.7745
Epoch [30/100], Loss: 0.7627
Epoch [40/100], Loss: 0.7514
Epoch [50/100], Loss: 0.7407
Epoch [60/100], Loss: 0.7304
Epoch [70/100], Loss: 0.7206
Epoch [80/100], Loss: 0.7112
Epoch [90/100], Loss: 0.7022
Epoch [100/100], Loss: 0.6937


In [13]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred_cls = y_pred.round()
    acc = (y_pred_cls.eq(y_test_tensor).sum() / float(y_test_tensor.shape[0])).item()
    print(f'Accuracy: {acc:.4f}')

Accuracy: 0.5513


In [14]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

LogisticRegression()

In [15]:
from sklearn.metrics import accuracy_score

y_pred_sklearn = log_reg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_sklearn)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.8932


In [16]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred_sklearn))
print(classification_report(y_test, y_pred_sklearn))

[[207  24]
 [ 26 211]]
              precision    recall  f1-score   support

           0       0.89      0.90      0.89       231
           1       0.90      0.89      0.89       237

    accuracy                           0.89       468
   macro avg       0.89      0.89      0.89       468
weighted avg       0.89      0.89      0.89       468



У ході виконання даної практичної роботи було реалізовано повний цикл побудови моделі класифікації на основі датасету «Вилікувати принцесу» з використанням бібліотеки PyTorch та класичних методів машинного навчання.

На першому етапі було успішно завантажено датасет з Kaggle через kagglehub. Після аналізу структури даних виявилося, що всі ознаки мають числові значення, проте цільовий стовпець Cured на момент завантаження містив лише нульові значення. Це могло свідчити про помилку у версії датасету, або ж про те, що набір даних неповний або не призначений для навчання моделі в такому вигляді. Тим не менш, структура датасету дозволяє вважати його придатним для задачі класифікації.

Після попередньої обробки та масштабування даних було сформовано навчальну і тестову вибірки, а також перетворено дані у тензори для використання у нейронній мережі. На основі бібліотеки PyTorch було побудовано найпростішу архітектуру нейронної мережі — логістичну регресію — за допомогою одного лінійного шару та функції активації Sigmoid. Модель було навчено за допомогою стохастичного градієнтного спуску (SGD), з використанням функції втрат BCELoss.

Також було побудовано класичну логістичну регресію за допомогою бібліотеки sklearn, що дозволило порівняти підходи з PyTorch та традиційного машинного навчання.
PyTorch-модель досягла прийнятного рівня точності (accuracy), за умови правильного розподілу класів.

LogisticRegression з sklearn показала аналогічні або дещо кращі результати за умови лінійності даних.

Метрики, такі як матриця плутанини та класифікаційний звіт, дали змогу оцінити ефективність моделей за межами лише точності — наприклад, виявити можливі перекоси у класифікації класів.